# 2a - Create Sentinel 2 time series for McKinley Mine seeding units
This notebook creates Sentinel 2 time series of NDVI or other band ratios for each seeding unit created by notebook 1.

In [ ]:
# Import libraries
import os
import datetime
from datetime import date
import configparser

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Import custom module for this project
from vegrestoretools import FisToDataframe, EvalScripts, FisClean

from sentinelhub import SHConfig, FisRequest, BBox, Geometry, CRS, WcsRequest, CustomUrlParam, \
    DataCollection, HistogramType
from sentinelhub.time_utils import iso_to_datetime, parse_time

In [ ]:
# Read SentinelHub configuration ID's from your local ini file
config_ini = configparser.ConfigParser()
config_ini.read('sentinel2_setup.ini')

# Populate a SentinelHub configuration object with the ID's from the local ini file
config = SHConfig()
config.instance_id = config_ini['Config']['instance_id'] # Instance ID for the Configuration
config.sh_client_id = config_ini['Config']['sh_client_id'] # Credentials from the OAuth client
config.sh_client_secret = config_ini['Config']['sh_client_secret']
config.save()

In [ ]:
# Other parameters
startDate = '2017-01-01'

In [ ]:
# Set working directory
os.chdir(os.path.join('D:/',
                      'McKinley'))

## Restore seeding unit geodataframe

In [ ]:
%store -r mckinley_seeding_subset_PARlo
mckinley_seeding_subset_PARlo

## Run SentinelHub Statistical Info Service (FIS) query

In [ ]:
seeding_aois = mckinley_seeding_subset_PARlo

# Convert seeding polygons to WGS84
seeding_aois = seeding_aois.to_crs("EPSG:4326")

# Explode multipolygons to polygons to query with FIS request
seeding_aois_exploded = seeding_aois.explode()

# Create empty list to store results for all polygons
output_list = []

# Loop over each polygon
for index, row in seeding_aois_exploded.iterrows():

    # TESTING - process specific seeding units
    if (index[0] == 172 or index[0] == 2650):

        # Status message
        print('Now processing Seeding Unit Index: ' + str(index[0]))

        # Setup parameters for FIS request
        geometry_list = Geometry(row.geometry, CRS.WGS84)
        time_interval = (startDate, date.today())

        # Retrieve custom script to pass to SentinelHub from vegrestoretools module
        evalscript = EvalScripts('Sentinel_2_NDVI')

        # Build FIS request
        fis_request = FisRequest(
            data_collection=DataCollection.SENTINEL2_L2A,
            layer='BANDS-S2-L2A',
            geometry_list=[geometry_list],
            time=time_interval,
            resolution='100m',
            custom_url_params={CustomUrlParam.EVALSCRIPT: evalscript},
            config=config
        )

        # Run the FIS request
        fis_data = fis_request.get_data()

        # Convert the FIS result to a dataframe
        fis_df = FisToDataframe(fis_data)

        # Add the seeding unit ID to the dataframe
        fis_df['ID'] = index[0]

        # Append the df to the temporary list
        output_list.append(fis_df)

    else:
        # Status message
        print('skipping this Seeding Unit')

# Concatenate the VMU results into a single DF
fis_df_all_raw = pd.concat(output_list, axis=0)
fis_df_all_raw

## Clean up the FIS results

In [ ]:
fis_df_all = FisClean(fis_df_all_raw)
fis_df_all

## Plot the results

In [ ]:
# Setup plot
fig, ax = plt.subplots(figsize=(15, 10))

# Build list of unique sites for legend
sites = fis_df_all.ID.unique()

fis_df_all.reset_index().groupby('ID').plot(x='date', y='mean', ax=ax)

# Legend and labels
plt.legend(sites)

plt.xlabel("Date", fontsize=20)
plt.ylabel("NDVI", fontsize=20)

ax.set_title(
    "Sentinel 2 NDVI time series for McKinley Mine Vegetation Management Units", fontsize=20)

plt.setp(ax.get_xticklabels(), rotation=90, horizontalalignment='right')

plt.show()